# Are we 3.12 yet?

Exploring the `conda-forge` [`python312` migration](https://conda-forge.org/status/#python312).

In [ ]:
import sys
if "pyodide" in sys.modules and __name__ == "__main__":
    print("installing dependencies...")
    %pip install -r requirements.txt

In [ ]:
if "pyodide" in sys.modules:
    print("patching http...")
    __import__("pyodide_http").patch_all()

In [ ]:
print("importing dependencies...")
import pandas as pd
from ipyforcegraph import graphs as G, behaviors as B
from ipyforcegraph.behaviors import scales as S 
import ipylab
import requests

In [ ]:
print("fetching data...")
URL = "https://raw.githubusercontent.com/regro/cf-graph-countyfair/master/status/migration_json/python312.json"
data = requests.get(URL).json()
data.keys()
df = pd.DataFrame([{"id": name, **d}for name, d in data.pop("_feedstock_status").items()]).fillna("")
df.sample(10).T

In [ ]:
print("building links...")
links = []
df.apply(lambda d: links.extend({"source": d.id, "target": t} for t in d.immediate_children), axis=1)
links = pd.DataFrame(links)
print("...link shape", links.shape)
links.T

In [ ]:
dk = [
    'bot-error',
    'not-solvable',
    'awaiting-parents',
    'unstable',
    'unknown',
    'awaiting-pr',
    'in-pr',
    'clean',
    'done',
]

In [ ]:
print("building nodes...")
nodes = pd.concat([df, pd.DataFrame(set(links["target"]) - set(df["id"]), columns=["id"])]).fillna("")
nodes["status"] = df.apply(lambda d: [k for k in data if d.id in data[k]][-1], axis=1)
nodes["status_color"] = df.apply(lambda d: [dk.index(k) for k in data if d.id in data[k]][-1], axis=1)
print('... node shape', nodes.shape)

In [ ]:
print("extracting pr nodes...")
pr_nodes = nodes[nodes["pr_status"] != ''][["pr_url", "id", "pr_status"]]
pr_nodes = pr_nodes.rename({'id': 'target', 'pr_url': 'id'}, axis=1)
pr_nodes["status"] = pr_nodes["pr_status"]
pr_nodes["status_color"] = pr_nodes.apply(lambda d: dk.index(d.pr_status), axis=1)
print("... pr nodes shape", pr_nodes.shape)
pr_nodes

In [ ]:
print("extracting PR links...")
pr_links = pr_nodes.rename({"id": "source"}, axis=1)
pr_links
print("...PR links shape", pr_links.shape)

In [ ]:
print('merging all nodes and links...')
all_nodes = pd.concat([nodes, pr_nodes[["id", "status", "status_color"]]]).fillna("")
all_links = pd.concat([links, pr_links[["source", "target", "status", "status_color"]]]).fillna("")
print('... nodes and links', all_nodes.shape, all_links.shape)

In [ ]:
print("initializing graph...")
fg = G.ForceGraph3D(
    source=G.DataFrameSource(nodes=all_nodes, links=all_links), 
    layout=dict(min_height="80vh")
)

In [ ]:
if __name__ == "__main__":
    app = ipylab.JupyterFrontEnd()
    panel = ipylab.Panel([fg])
    app.shell.add(panel, "main", {"mode": "split-right"})

pretty

In [ ]:
print("adding pretties...")
c = S.ContinuousColor(
    "status_color", scheme=S.ContinuousColor.Scheme.viridis, domain=[-2, 9]
)
s = 4
ns = B.NodeShapes(color=B.CaptureAs("_color", c), size=s)
tip = B.NodeTooltip(B.Nunjucks("""
<h1>{{ node.id }}</h1> ({{ node.status }})
{{ node.immediate_children }}
"""), color="black")

ls = B.LinkShapes(
    color=B.ReplaceCssVariables(
        B.Nunjucks("""
        var({% if link.status %}
            {{ {
                "unstable": "--jp-error-color0", 
                "unknown": "--jp-warn-color0", 
                "clean": "--jp-brand-color0"
                }[link.status] }}
            {%- else -%}
            --jp-layout-color4
            {%- endif -%}
        )
        """)
    ),
    width=B.Nunjucks("""
        {% if link.status_color %}15{% else %}1{% endif %}
    """),
    opacity=0.1
)
fg.behaviors=[ns, tip, ls]